## Data Import

In [26]:
import pandas as pd
import numpy as np
import re
import os
import glob
import shutil

import boto3

import os
import time
import boto3
import numpy as np
import pandas as pd
import sagemaker
from sagemaker import get_execution_role
from sagemaker.workflow.pipeline_context import PipelineSession


!pip install --upgrade pip
%conda install tensorflow 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.5.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


### Data Import for Interactions

In [27]:
# role = sagemaker.get_execution_role()
# region = boto3.Session().region_name

# data_bucket = "dynamodbpetfinder"
# object_prefix = "interaction_"
# local_path = "data/interaction/"

# if not os.path.exists(local_path):
#     os.makedirs(local_path)
# else:
#     shutil.rmtree(local_path)
#     os.makedirs(local_path)

# pattern = r'[0-9]+'
# s3 = boto3.client("s3")

# result = s3.list_objects(Bucket=data_bucket, Prefix=object_prefix, Delimiter="/")
# subfolders = [re.search(pattern, o.get("Prefix")).group() for o in result.get("CommonPrefixes")]
# subfolders.sort(reverse=True)

# object_path = object_prefix + subfolders[0] + "/"
# files = s3.list_objects(Bucket=data_bucket, Prefix=object_path, Delimiter="/")

# pattern = rf'{object_path}(.+)'
# for content in files.get("Contents"):
#     file_path = content.get("Key")
#     filename = re.findall(pattern, file_path)[0]
#     print(filename)

#     with open(local_path+filename, 'wb') as file:
#         s3.download_fileobj(
#             Bucket=data_bucket,
#             Key=file_path,
#             Fileobj=file
#         )

In [28]:
# file_list = glob.glob(local_path + "*")

# print(file_list)

# dfs = [] # an empty list to store the data frames
# for file in file_list:
#     data = pd.read_csv(file) # read data frame from csv file
#     dfs.append(data) # append the data frame to the list
    
# interactions = pd.concat(dfs, ignore_index=True)

## Data Import for Cats

In [29]:
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

data_bucket = "dynamodbpetfinder"
object_prefix = "cat_"
local_path = "data/cat/"

if not os.path.exists(local_path):
    os.makedirs(local_path)
else:
    shutil.rmtree(local_path)
    os.makedirs(local_path)

pattern = r'[0-9]+'
s3 = boto3.client("s3")

result = s3.list_objects(Bucket=data_bucket, Prefix=object_prefix, Delimiter="/")
subfolders = [re.search(pattern, o.get("Prefix")).group() for o in result.get("CommonPrefixes")]
subfolders.sort(reverse=True)

object_path = object_prefix + subfolders[0] + "/"
files = s3.list_objects(Bucket=data_bucket, Prefix=object_path, Delimiter="/")

pattern = rf'{object_path}(.+)'
for content in files.get("Contents"):
    file_path = content.get("Key")
    filename = re.findall(pattern, file_path)[0]
    print(filename)

    with open(local_path+filename, 'wb') as file:
        s3.download_fileobj(
            Bucket=data_bucket,
            Key=file_path,
            Fileobj=file
        )

run-1686726820579-part-r-00000
run-1686726820579-part-r-00001
run-1686726820579-part-r-00002
run-1686726820579-part-r-00003


In [30]:
file_list = glob.glob(local_path + "*")

print(file_list)

dfs = [] # an empty list to store the data frames
for file in file_list:
    data = pd.read_csv(file) # read data frame from csv file
    dfs.append(data) # append the data frame to the list
    
cats = pd.concat(dfs, ignore_index=True)

['data/cat/run-1686726820579-part-r-00003', 'data/cat/run-1686726820579-part-r-00001', 'data/cat/run-1686726820579-part-r-00000', 'data/cat/run-1686726820579-part-r-00002']


In [31]:
cats.head()

,id,cws_id,C_othercats,C_kids,C_primarycolor,name,C_employment,sweet,C_homeownership,active,...,quiet,C_otherdogs,C_attention,loving,updatedAt,C_adoptionfee,C_sweetspicy,description,spicy,C_energy
0,54b908dd-a325-459a-bdcb-5b1901d8324e,1681491254-120,-1,0,White,BoyBoy the oriental mix tabby cat,no preference,True,owned,False,...,True,0,independent,True,2023-05-15T02:51:11.735Z,yes,sweet,"quiet, loving, sweet, shy",False,chill
1,1a929522-33b0-42f2-8a80-a6f097a0e1fe,1681491547-251,1,0,Tabby,Mike and Bella,working full time,True,no preference,True,...,False,0,NaN,True,2023-05-15T02:51:14.961Z,no preference,sweet,"playful, active, loving, sweet",False,high energy
2,b3538eec-3e8b-4c48-9adf-08c5c5fbbfce,1681491592-271,0,0,White,Maya for adoption,student,False,parents,False,...,False,0,NaN,False,2023-05-15T02:51:19.367Z,yes,NaN,NaN,False,chill
3,b79b5e65-621b-4906-aea4-fffef2259fec,1681491440-203,1,0,Calico/Tortie,Sally – 8 mths old tortoiseshell,working full time,False,owned,False,...,True,0,independent,True,2023-05-15T02:51:36.655Z,no preference,sweet,"quiet, loving, friendly",False,chill
4,6117e9df-fa0f-43dd-b6cb-b62a8748bfd8,1681491915-414,0,0,Tabby,Matcha,working full time,True,owned,False,...,False,0,NaN,True,2023-05-15T02:51:50.223Z,yes,sweet,"loving, sweet",False,chill


### Data Import for Users

In [32]:
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

data_bucket = "dynamodbpetfinder"
object_prefix = "user_"
local_path = "data/user/"

if not os.path.exists(local_path):
    os.makedirs(local_path)
else:
    shutil.rmtree(local_path)
    os.makedirs(local_path)

pattern = r'[0-9]+'
s3 = boto3.client("s3")

result = s3.list_objects(Bucket=data_bucket, Prefix=object_prefix, Delimiter="/")
subfolders = [re.search(pattern, o.get("Prefix")).group() for o in result.get("CommonPrefixes")]
subfolders.sort(reverse=True)

object_path = object_prefix + subfolders[0] + "/"
files = s3.list_objects(Bucket=data_bucket, Prefix=object_path, Delimiter="/")

pattern = rf'{object_path}(.+)'
for content in files.get("Contents"):
    file_path = content.get("Key")
    filename = re.findall(pattern, file_path)[0]
    print(filename)

    with open(local_path+filename, 'wb') as file:
        s3.download_fileobj(
            Bucket=data_bucket,
            Key=file_path,
            Fileobj=file
        )

run-1686726799591-part-r-00000
run-1686726799591-part-r-00001
run-1686726799591-part-r-00002
run-1686726799591-part-r-00003


In [33]:
file_list = glob.glob(local_path + "*")

print(file_list)

dfs = [] # an empty list to store the data frames
for file in file_list:
    data = pd.read_json(file, lines=True) # read data frame from json file
    dfs.append(data) # append the data frame to the list
    
users = pd.concat(dfs, ignore_index=True)


['data/user/run-1686726799591-part-r-00002', 'data/user/run-1686726799591-part-r-00000', 'data/user/run-1686726799591-part-r-00003', 'data/user/run-1686726799591-part-r-00001']


In [34]:
users.head()

,id,__typename,A_othercats,A_sweetspicy,_lastChangedAt,A_agegroup,A_gender,A_otherdogs,A_employment,createdAt,...,A_firstcat,A_kids,_version,A_attention,updatedAt,A_primarycolor,username,A_homeownership,A_allergies,A_energy
0,b2fe504c-8b80-4f50-ab07-12b6e7ff8cd2,User,-1,Anything is nice,1686310110279,"[{'S': 'Kitten'}, {'S': 'Juvenile'}]",No preference,-1,Student,2023-06-09T11:28:30.250Z,...,1,-1,1,Independent,2023-06-09T11:28:30.250Z,[{'S': 'No preference'}],gadmantang,Staying with Parents,-1,Chill
1,079b0ec9-cec6-42fb-9f00-7891c52a10fb,User,-1,All sweet,1684430025291,[{'S': 'Juvenile'}],No preference,-1,Working Full Time,2023-05-18T17:13:45.262Z,...,1,-1,1,No preference,2023-05-18T17:13:45.262Z,[{'S': 'No preference'}],Goodboya,Self-Owned,-1,No preference
2,6abb6324-ae5e-48d5-a55f-0134f21c79c2,User,-1,Some spice,1684409930581,"[{'S': 'No preference'}, {'S': 'Kitten'}, {'S'...",Male,-1,Student,2023-05-18T11:38:50.554Z,...,1,-1,1,Independent,2023-05-18T11:38:50.554Z,[{'S': 'No preference'}],John Doe,Staying with Parents,-1,Chill
3,fab5c53d-493a-4baf-b8e5-3cd1e505d75e,User,-1,All sweet,1685623771616,"[{'S': 'No preference'}, {'S': 'Kitten'}]",No preference,-1,Working Full Time,2023-06-01T12:49:31.590Z,...,1,-1,1,An attention seeker,2023-06-01T12:49:31.590Z,"[{'S': 'No preference'}, {'S': 'Calico/Tortie'}]",Testing :),Self-Owned,-1,No preference
4,6c66277d-fc3b-4d17-9c90-f1f37579109a,User,-1,All sweet,1684431886401,[{'S': 'Kitten'}],Female,1,Student,2023-05-18T17:44:46.373Z,...,1,1,1,Independent,2023-05-18T17:44:46.373Z,[{'S': 'No preference'}],ridhicar,Staying with Parents,-1,High energy


## Define a Processing Step for Feature Engineering

### Preprocessing for Users

In [35]:
# rename headers
cl_users = users.rename(columns={'A_gender': 'gender',
                              'A_primarycolor': 'primary_color',
                              'A_agegroup': 'age_group',
                              'A_energy': 'energy_level',
                              'A_attention': 'attention_need',
                              'A_sweetspicy': 'personality',
                              'A_firstcat': 'is_first_cat',
                              'A_othercats': 'has_other_cats',
                              'A_otherdogs': 'good_with_other_dogs',
                              'A_kids': 'good_with_kids',
                              'A_employment': 'employment',
                              'A_homeownership': 'home_ownership',
                              'A_allergies': 'has_allergies',
                              'A_adoptionfee': 'agree_to_fee',
                              'createdAt': 'created_at',
                              'updatedAt': 'updated_at',
                              })

# clean multi-select columns with No Preference options (age, color)

def clean_multi_select(row):
    arr = [o.get("S") for o in row]
    if (len(arr) > 1) and ('No preference' in arr):
        arr.remove('No preference')
    return [s.lower() for s in arr]

cl_users['age_group'] = cl_users['age_group'].map(lambda choice: clean_multi_select(choice))
cl_users['primary_color'] = cl_users['primary_color'].map(lambda choice: clean_multi_select(choice))


# split columns with list (age, color)
age_groups = cl_users['age_group'].explode().unique().tolist()
split_age_groups = cl_users['age_group'].map(lambda row: ','.join([str(age in row) for age in age_groups]))
new_age_columns = split_age_groups.str.split(',', expand=True)
new_age_columns = new_age_columns.applymap(lambda val: (val == 'True'))
# new_age_columns = new_age_columns.astype('bool')
new_age_columns = new_age_columns.astype('int')

pattern = re.compile(r'\s|/')
# new_age_columns.columns = [f'age_{pattern.sub("_", age).lower()}' for age in age_groups]
cl_users[[f'age_{pattern.sub("_", age).lower()}' for age in age_groups]] = new_age_columns
cl_users = cl_users.drop('age_group', axis=1)

color_groups = cl_users['primary_color'].explode().unique().tolist()
split_color_groups = cl_users['primary_color'].map(lambda row: ','.join([str(color in row) for color in color_groups]))
new_color_columns = split_color_groups.str.split(',', expand=True)
new_color_columns = new_color_columns.applymap(lambda val: (val == 'True'))
# new_color_columns = new_color_columns.astype('bool')
new_color_columns = new_color_columns.astype('int')

cl_users[[f'primary_color_{pattern.sub("_", color).lower()}' for color in color_groups]] = new_color_columns
cl_users = cl_users.drop('primary_color', axis=1)


# convert string fields to lower case (gender, energy_level, attention_need, personality, employment, home_ownership)
cl_users['gender'] = cl_users['gender'].map(lambda val: val.lower())
cl_users['energy_level'] = cl_users['energy_level'].map(lambda val: val.lower())
cl_users['attention_need'] = cl_users['attention_need'].map(lambda val: val.lower())
cl_users['personality'] = cl_users['personality'].map(lambda val: val.lower())
cl_users['employment'] = cl_users['employment'].map(lambda val: val.lower())
cl_users['home_ownership'] = cl_users['home_ownership'].map(lambda val: val.lower())



# # convert int booleans to actual booleans (is_first_cat, has_other_cats, good_with_other_dogs, good_with_kids, has_allergies, agree_to_fee)
cl_users['is_first_cat'] = cl_users['is_first_cat'].map(lambda val: (val == 1))
cl_users['has_other_cats'] = cl_users['has_other_cats'].map(lambda val: (val == 1))
cl_users['good_with_other_dogs'] = cl_users['good_with_other_dogs'].map(lambda val: (val == 1))
cl_users['good_with_kids'] = cl_users['good_with_kids'].map(lambda val: (val == 1))
cl_users['has_allergies'] = cl_users['has_allergies'].map(lambda val: (val == 1))
cl_users['agree_to_fee'] = cl_users['agree_to_fee'].map(lambda val: (val == 1))

# convert types
cl_users['created_at'] = pd.to_datetime(cl_users['created_at']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')
cl_users['updated_at'] = pd.to_datetime(cl_users['updated_at']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')

cl_users = cl_users.astype({
                          'is_first_cat': 'int',
                          'has_other_cats': 'int',
                          'good_with_other_dogs': 'int',
                          'good_with_kids': 'int',
                          'has_allergies': 'int',
                          'agree_to_fee': 'int',
                          'created_at': 'object',
                          'updated_at': 'object',
                          })

# drop glue columns
cl_users = cl_users.drop(['__typename', '_lastChangedAt', '_version'], axis=1)

cl_users = cl_users.set_index('id')

## Export Processed Data as csv to Local Folder
processed_folder = "processed/auxiliary/"

if not os.path.exists(processed_folder):
    os.makedirs(processed_folder)

cl_users.to_csv(f'{processed_folder}users.csv')

### Preprocessing for Cats

In [36]:
%conda install tensorflow
%pip install -U opencv-python-headless
import cv2
import tensorflow as tf

# rename headers
cl_cats = cats.rename(columns={'C_gender': 'gender',
                              'C_primarycolor': 'primary_color',
                              'C_agegroup': 'age_group',
                              'C_energy': 'energy_level',
                              'C_attention': 'attention_need',
                              'C_sweetspicy': 'personality',
                              'C_firstcat': 'good_first_cat',
                              'C_othercats': 'good_with_other_cats',
                              'C_otherdogs': 'good_with_other_dogs',
                              'C_kids': 'good_with_kids',
                              'C_employment': 'preferred_employment',
                              'C_homeownership': 'preferred_home_ownership',
                              'C_allergies': 'good_with_allergies',
                              'C_adoptionfee': 'require_fee',
                              'createdAt': 'created_at',
                              'updatedAt': 'updated_at',
                              })


# fill attention and personality columns as neutral
cl_cats['attention_need'] = cl_cats['attention_need'].fillna('neutral')
cl_cats['personality'] = cl_cats['personality'].fillna('neutral')

# fill empty description with no description available
cl_cats['description'] = cl_cats['description'].fillna('no description available')



# map integers in categorical columns
def map_categorical(val):
    if val == 1:
        return 'yes'
    elif val == -1:
        return 'no'
    else:
        return 'neutral'

cl_cats['good_first_cat'] = cl_cats['good_first_cat'].map(lambda val: map_categorical(val))
cl_cats['good_with_other_cats'] = cl_cats['good_with_other_cats'].map(lambda val: map_categorical(val))
cl_cats['good_with_other_dogs'] = cl_cats['good_with_other_dogs'].map(lambda val: map_categorical(val))
cl_cats['good_with_kids'] = cl_cats['good_with_kids'].map(lambda val: map_categorical(val))

# convert string fields to lower case (gender, breed, primary_color, age_group, energy_level, attention_need, personality, preferred_employment, preferred_home_ownership, require_fee)
cl_cats['gender'] = cl_cats['gender'].map(lambda val: val.lower())
cl_cats['breed'] = cl_cats['breed'].map(lambda val: val.lower())
cl_cats['primary_color'] = cl_cats['primary_color'].map(lambda val: val.lower())
cl_cats['age_group'] = cl_cats['age_group'].map(lambda val: val.lower())
cl_cats['energy_level'] = cl_cats['energy_level'].map(lambda val: val.lower())
cl_cats['attention_need'] = cl_cats['attention_need'].map(lambda val: val.lower())
cl_cats['personality'] = cl_cats['personality'].map(lambda val: val.lower())
cl_cats['preferred_employment'] = cl_cats['preferred_employment'].map(lambda val: val.lower())
cl_cats['preferred_home_ownership'] = cl_cats['preferred_home_ownership'].map(lambda val: val.lower())
cl_cats['require_fee'] = cl_cats['require_fee'].map(lambda val: val.lower())

cl_cats['description'] = cl_cats['description'].map(lambda val: val.encode("ascii", "ignore").decode().replace('\n', ' ').replace('\r', '').lower())
cl_cats['details'] = cl_cats['details'].map(lambda val: val.encode("ascii", "ignore").decode().replace('\n', ' ').replace('\r', '').lower())

# convert types
cl_cats['created_at'] = pd.to_datetime(cl_cats['created_at']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')
cl_cats['updated_at'] = pd.to_datetime(cl_cats['updated_at']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')

cl_cats = cl_cats.astype({'playful': 'int',
                          'active': 'int',
                          'curious': 'int',
                          'talkative': 'int',
                          'quiet': 'int',
                          'loving': 'int',
                          'sweet': 'int',
                          'likes_held': 'int',
                          'friendly': 'int',
                          'shy': 'int',
                          'spicy': 'int',
                          'loves_attention': 'int'
                          })

# drop glue columns
cl_cats = cl_cats.drop(['__typename', '_lastChangedAt', '_version'], axis=1)

## Export Processed Data as csv to Local Folder
processed_folder = "processed/auxiliary/"

if not os.path.exists(processed_folder):
    os.makedirs(processed_folder)

cl_cats.to_csv(f'{processed_folder}cats.csv')

## cat profile images

new_size = 128

data_bucket = "petfinder6000images"
local_path = "data/cat_images/"

if not os.path.exists(local_path):
    os.makedirs(local_path)
else:
    shutil.rmtree(local_path)
    os.makedirs(local_path)
    
s3 = boto3.client("s3")



def vectorise(cws_id):
    filename = f'cropped_{cws_id}.jpg'
    s3.download_file(
        Bucket=data_bucket,
        Key=filename,
        Filename=local_path+filename
    )
    
    img = cv2.imread(local_path+filename)
    try:
        resize = cv2.resize(img, (new_size, new_size))
        resize = cv2.cvtColor(resize,cv2.COLOR_BGR2RGB)
        resize = np.array(resize.tolist())/255.0
        return resize
        # img_flat = resize.reshape(-1)
        # return str(np.array2string(img_flat, precision=2, separator=',', suppress_small=True)).encode('utf-8')
    except:
        print(file)

cat_images = cl_cats.loc[:, ['cws_id', 'updated_at']]
cat_images['img_shape'] = str([new_size, new_size, 3])
cat_images['img_vector'] = cat_images['cws_id'].map(lambda cws_id: vectorise(cws_id))

img_vectors = cat_images['img_vector'].values



# Using a pretrained model to generate features

#### Importing pretrained MobileNetV2
mnetv2_base = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(new_size,new_size,3), include_top=False, weights='imagenet')

# Freezing layers
for layer in mnetv2_base.layers:
    layer.trainable = False

    
# defining a function to extract features
def img_feature_extraction(img_vectors, pre_model):

    batch_img = []
    
    # preprocessing and then using pretrained model to extract features
    for image in img_vectors:
        im_toarray = tf.keras.preprocessing.image.img_to_array(image)
        
        im_toarray = np.expand_dims(image, axis=0)
        im_toarray = tf.keras.applications.mobilenet.preprocess_input(im_toarray)
        
        batch_img.append(im_toarray)
        
    batch_img = np.vstack(batch_img)
    features = pre_model.predict(batch_img, batch_size=64)
    features= features.reshape((len(img_vectors), -1))
    return features



# Extract features
features = img_feature_extraction(img_vectors, mnetv2_base)

features.shape

# add the feature vectors to dataframe
feature_vectors = pd.Series(features.tolist(), index = cat_images.index)
cat_images = cat_images.merge(feature_vectors.rename('feature_vectors'), left_index=True, right_index=True)

# stringify vectors

cat_images['feature_vectors']=cat_images['feature_vectors'].astype(pd.StringDtype())

cat_images['img_vector']=cat_images['img_vector'].apply(lambda x: x.reshape(-1))
cat_images['img_vector']=cat_images['img_vector'].astype(pd.StringDtype())



def cast_object_to_string(data_frame):
    """
    Cast all columns of data_frame of type object to type string and return it.
    Parameters:
        data_frame: A pandas Dataframe
    Returns:
        Data frame
    """
    for label in data_frame.columns:
        if data_frame.dtypes[label] == object:
            data_frame[label] = data_frame[label].astype("str").astype("string")
    return data_frame

cat_images = cast_object_to_string(cat_images)

cat_images = cat_images.reset_index()


processed_folder = "processed/auxiliary/"

if not os.path.exists(processed_folder):
    os.makedirs(processed_folder)

cat_images.to_pickle(f'{processed_folder}cat-images.pkl')


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.5.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [37]:

#     # rename headers
#     cl_interactions = interactions.rename(columns={
#                                   'createdAt': 'created_at',
#                                   'updatedAt': 'updated_at',
#                                   })

#     # convert types
#     cl_interactions = cl_interactions.astype({
#                               'like': 'int',
#                               'click': 'int',
#                               })
#     cl_interactions['created_at'] = pd.to_datetime(cl_interactions['created_at']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')
#     cl_interactions['updated_at'] = pd.to_datetime(cl_interactions['updated_at']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')

#     # drop glue columns
#     cl_interactions = cl_interactions.drop(['__typename', '_lastChangedAt', '_version'], axis=1)

#     # exporting processed data as csv to local folder

#     processed_folder = "processed/auxiliary/"

#     if not os.path.exists(processed_folder):
#         os.makedirs(processed_folder)

#     cl_interactions.to_csv(f'{processed_folder}interactions.csv')
  
#     ## train-test split 

#     from sklearn.model_selection import train_test_split

#     test_size = 0.2
#     validation_size = 0.5 # of test size
#     random_state = 2023

#     ## leave some users out
#     data_dir = os.path.join(os.getcwd(), "data")
#     os.makedirs(data_dir, exist_ok=True)

#     output_dir = os.path.join(os.getcwd(), "data/output")
#     os.makedirs(output_dir, exist_ok=True)
    
#     users = cl_interactions['userID'].unique()
    
#     train_users, test_users = train_test_split(users, test_size=test_size, shuffle=True, random_state=random_state)
#     validation_users, test_users = train_test_split(test_users, test_size=validation_size, shuffle=True, random_state=random_state)

#     train_set = cl_interactions[cl_interactions['userID'].isin(train_users)]
#     validation_set = cl_interactions[cl_interactions['userID'].isin(validation_users)]
#     test_set = cl_interactions[cl_interactions['userID'].isin(test_users)]
#     print(train_set)
#     train_set.to_csv('../data/output/lsuo_train.csv')
#     validation_set.to_csv('../data/output/lsuo_validation.csv')
#     test_set.to_csv('../data/output/lsuo_test.csv')

#     ## stratified split

#     train_set = cl_interactions.groupby('userID').sample(frac=1-test_size, random_state=random_state)
#     test_set = cl_interactions.drop(train_set.index)

#     validation_set = test_set.groupby('userID').sample(frac=validation_size, random_state=random_state)
#     test_set = test_set.drop(validation_set.index)

#     print(f'Total number of users: {cl_interactions["userID"].nunique()}')
#     print(f'Number of users in training: {train_set["userID"].nunique()}')
#     print(f'Number of users in validation: {validation_set["userID"].nunique()}')
#     print(f'Number of users in test: {test_set["userID"].nunique()}')

#     train_set.to_csv('../data/output/strat_train.csv')
#     validation_set.to_csv('../data/output/strat_validation.csv')
#     test_set.to_csv('../data/output/strat_test.csv')

In [38]:
!mkdir -p code

In [39]:
%%writefile code/stratified-split.py

# !pip install sagemaker
# %conda install tensorflow
# %pip install -U opencv-python-headless

import glob
import numpy as np
import pandas as pd
import os
import json
import joblib
from io import StringIO
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tarfile
import boto3
import re

import shutil
import time



try:
    from sagemaker_containers.beta.framework import (
        content_types,
        encoders,
        env,
        modules,
        transformer,
        worker,
        server,
    )
except ImportError:
    pass

if __name__ == "__main__":

    # role = sagemaker.get_execution_role()
    region = boto3.Session().region_name

    data_bucket = "dynamodbpetfinder"
    object_prefix = "interaction_"
    local_path = "data/interaction/"

    if not os.path.exists(local_path):
        os.makedirs(local_path)
    else:
        shutil.rmtree(local_path)
        os.makedirs(local_path)

    pattern = r'[0-9]+'
    s3 = boto3.client("s3")

    result = s3.list_objects(Bucket=data_bucket, Prefix=object_prefix, Delimiter="/")
    subfolders = [re.search(pattern, o.get("Prefix")).group() for o in result.get("CommonPrefixes")]
    subfolders.sort(reverse=True)

    object_path = object_prefix + subfolders[0] + "/"
    files = s3.list_objects(Bucket=data_bucket, Prefix=object_path, Delimiter="/")

    pattern = rf'{object_path}(.+)'
    for content in files.get("Contents"):
        file_path = content.get("Key")
        filename = re.findall(pattern, file_path)[0]
        print(filename)

        with open(local_path+filename, 'wb') as file:
            s3.download_fileobj(
                Bucket=data_bucket,
                Key=file_path,
                Fileobj=file
            )
 
    file_list = glob.glob(local_path + "*")

    print(file_list)

    dfs = [] # an empty list to store the data frames
    for file in file_list:
        data = pd.read_csv(file) # read data frame from csv file
        dfs.append(data) # append the data frame to the list

    interactions = pd.concat(dfs, ignore_index=True)

      # rename headers
    cl_interactions = interactions.rename(columns={
                                  'createdAt': 'created_at',
                                  'updatedAt': 'updated_at',
                                  })

    # convert types
    cl_interactions = cl_interactions.astype({
                              'like': 'int',
                              'click': 'int',
                              })
    cl_interactions['created_at'] = pd.to_datetime(cl_interactions['created_at']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')
    cl_interactions['updated_at'] = pd.to_datetime(cl_interactions['updated_at']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')

    # drop glue columns
    cl_interactions = cl_interactions.drop(['__typename', '_lastChangedAt', '_version'], axis=1)

    # exporting processed data as csv to local folder

    processed_folder = "processed/auxiliary/"

    if not os.path.exists(processed_folder):
        os.makedirs(processed_folder)

    cl_interactions.to_csv(f'{processed_folder}interactions.csv')


    ## train-test split 

    from sklearn.model_selection import train_test_split

    test_size = 0.2
    validation_size = 0.5 # of test size
    random_state = 2023

    ## leave some users out

    users = cl_interactions['userID'].unique()

    train_users, test_users = train_test_split(users, test_size=test_size, shuffle=True, random_state=random_state)
    validation_users, test_users = train_test_split(test_users, test_size=validation_size, shuffle=True, random_state=random_state)

    train_set = cl_interactions[cl_interactions['userID'].isin(train_users)]
    validation_set = cl_interactions[cl_interactions['userID'].isin(validation_users)]
    test_set = cl_interactions[cl_interactions['userID'].isin(test_users)]

    output_dir = os.path.join(os.getcwd(), "data/output")
    os.makedirs(output_dir, exist_ok=True)
    
    train_set.to_csv('../data/output/lsuo_train.csv')
    validation_set.to_csv('../data/output/lsuo_validation.csv')
    test_set.to_csv('../data/output/lsuo_test.csv')

    ## stratified split

    train_set = cl_interactions.groupby('userID').sample(frac=1-test_size, random_state=random_state)
    test_set = cl_interactions.drop(train_set.index)

    validation_set = test_set.groupby('userID').sample(frac=validation_size, random_state=random_state)
    test_set = test_set.drop(validation_set.index)

    print(f'Total number of users: {cl_interactions["userID"].nunique()}')
    print(f'Number of users in training: {train_set["userID"].nunique()}')
    print(f'Number of users in validation: {validation_set["userID"].nunique()}')
    print(f'Number of users in test: {test_set["userID"].nunique()}')

    train_set.to_csv('../data/output/strat_train.csv')
    validation_set.to_csv('../data/output/strat_validation.csv')
    test_set.to_csv('../data/output/strat_test.csv')




Writing code/stratified-split.py


In [40]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.session import Session

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
bucket_name = "petfinder6000-training"
base_job_name = "strat"

sklearn_framework_version = "1.2-1"

sklearn_processor = SKLearnProcessor(
    framework_version=sklearn_framework_version,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    base_job_name=base_job_name,
    role=role,
    sagemaker_session=Session(default_bucket=bucket_name),
)

processed_folder = "./processed/auxiliary/"

processor_args = sklearn_processor.run(
    code="code/stratified-split.py",
    # arguments = ["arg1", "arg2"], # Arguments can optionally be specified here
    inputs=[],
    outputs=[
        ProcessingOutput(source="/opt/ml/processing/train", output_name="train"),
        ProcessingOutput(source="/opt/ml/processing/validation", output_name="validation"),
        ProcessingOutput(source="/opt/ml/processing/test", output_name="test"),
    ],
)

INFO:sagemaker:Creating processing-job with name strat-2023-06-14-09-16-24-757


......................run-1686726857238-part-r-00000
run-1686726857238-part-r-00001
run-1686726857238-part-r-00002
run-1686726857238-part-r-00003
['data/interaction/run-1686726857238-part-r-00000', 'data/interaction/run-1686726857238-part-r-00003', 'data/interaction/run-1686726857238-part-r-00002', 'data/interaction/run-1686726857238-part-r-00001']
Total number of users: 104
Number of users in training: 104
Number of users in validation: 96
Number of users in test: 100



In [41]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

step_process = ProcessingStep(
    name="PreprocessData",
    processor=sklearn_processor,  # Provide the processor object here
    code="code/stratified-split.py",  # Specify the code to be run
    inputs=[],  # Provide your inputs
    outputs=[  # Specify your outputs
        ProcessingOutput(source="/opt/ml/processing/train", output_name="train"),
        ProcessingOutput(source="/opt/ml/processing/validation", output_name="validation"),
        ProcessingOutput(source="/opt/ml/processing/test", output_name="test"),
    ]
)

In [52]:
interactions

NameError: name 'interactions' is not defined

## Feature Store

In [42]:
import pandas as pd
import numpy as np
import re
import os
import glob
import shutil

import boto3
import sagemaker
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup



In [43]:
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
boto_session = boto3.Session(region_name=region)


In [44]:
bucket = "petfinder6000-auxiliary"
prefix = "sagemaker-featurestore"
offline_feature_store_bucket = 's3://{}/{}'.format(bucket, prefix)
print(f'Offline bucket: {offline_feature_store_bucket}')

Offline bucket: s3://petfinder6000-auxiliary/sagemaker-featurestore


In [45]:
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)


In [46]:
## Create Feature Group

In [47]:
# CHANGE HERE!!
data_type = 'auxiliary'
data_name = 'cat-images'
file_type = 'pkl'

In [48]:
feature_group_name = f'{data_name}-feature-group'
feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=feature_store_session)

In [49]:
# load data from csv
processed_dir = 'processed'
if file_type == 'csv':
    data = pd.read_csv(f'{processed_dir}/{data_type}/{data_name}.{file_type}', header=0)
else:
    data = pd.read_pickle(f'{processed_dir}/{data_type}/{data_name}.{file_type}')

In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            404 non-null    int64 
 1   cws_id           404 non-null    string
 2   updated_at       404 non-null    string
 3   img_shape        404 non-null    string
 4   img_vector       404 non-null    string
 5   feature_vectors  404 non-null    string
dtypes: int64(1), string(5)
memory usage: 19.1 KB


In [51]:
record_identifier_name = "id"
event_time_feature_name = "updated_at"
feature_group.load_feature_definitions(data_frame=data)

feature_group.create(
    s3_uri=offline_feature_store_bucket,
    record_identifier_name=record_identifier_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=False
)


ClientError: An error occurred (ValidationException) when calling the CreateFeatureGroup operation: Validation Error: RecordIdentifierFeatureName is not found in names of FeatureDefinitions.

In [ ]:
# check status of feature group creation
status = feature_group.describe().get("FeatureGroupStatus")
print(status)


In [ ]:
# load data into feature group
feature_group.ingest(data_frame=data, max_workers=3, wait=True)



In [ ]:
sagemaker_client.list_feature_groups()


In [ ]:
# # to remove feature groups
# sagemaker_session = sagemaker.Session()
# feature_group_name = "cats-feature-group"
# feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=sagemaker_session)
# feature_group.delete() 
